In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sqlite3

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/sakila/sakila.db


## 1. Excercises for Sakila database from [Sakila Queries Excercise](https://datamastery.gitlab.io/exercises/sakila-queries.html)
#### I am using sqlite3 version of Sakila database available here [Sakila sqlite](https://github.com/ivanceras/sakila/tree/master/sqlite-sakila-db)

In [21]:
# connection test and first query

conn = sqlite3.connect("/kaggle/input/sakila/sakila.db")

pd.read_sql("""
SELECT
    *
FROM
    staff;
""", conn)

,staff_id,first_name,last_name,address_id,picture,email,store_id,active,username,password,last_update
0,1,Mike,Hillyer,3,None,Mike.Hillyer@sakilastaff.com,1,1,Mike,8cb2237d0679ca88db6464eac60da96345513964,2011-09-14 18:05:32
1,2,Jon,Stephens,4,None,Jon.Stephens@sakilastaff.com,2,1,Jon,8cb2237d0679ca88db6464eac60da96345513964,2011-09-14 18:05:32


In [22]:
# function for quick access do db
def q(sql_query):
    return pd.read_sql(sql_query, conn)

In [23]:
# Which actors have the first name ‘Scarlett’
q(
"""
SELECT
    actor_id,
    first_name,
    last_name
FROM
    actor
WHERE first_name = "SCARLETT"
""")

,actor_id,first_name,last_name
0,81,SCARLETT,DAMON
1,124,SCARLETT,BENING


In [24]:
# Which actors have the last name ‘Johansson’
q(
"""
SELECT
    actor_id,
    first_name,
    last_name
FROM
    actor
WHERE last_name = "JOHANSSON"
""")

,actor_id,first_name,last_name
0,8,MATTHEW,JOHANSSON
1,64,RAY,JOHANSSON
2,146,ALBERT,JOHANSSON


In [25]:
# How many distinct actors last names are there?
q(
"""
SELECT
    COUNT(DISTINCT(last_name)) num_of_distinct_ln
FROM
    actor
""")

,num_of_distinct_ln
0,121


In [26]:
# Which last names are not repeated?
q(
"""
SELECT
    last_name
FROM
    actor
GROUP BY last_name
HAVING COUNT(last_name) =1
""")

,last_name
0,ASTAIRE
1,BACALL
2,BALE
3,BALL
4,BARRYMORE
...,...
61,WALKEN
62,WAYNE
63,WILSON
64,WITHERSPOON


In [27]:
# Which last names appear more than once?
q(
"""
SELECT
    last_name
FROM
    actor
GROUP BY last_name
HAVING COUNT(last_name) >1
""")

,last_name
0,AKROYD
1,ALLEN
2,BAILEY
3,BENING
4,BERRY
5,BOLGER
6,BRODY
7,CAGE
8,CHASE
9,CRAWFORD


In [28]:
# Which actor has appeared in the most films?
q(
"""
SELECT
    a.first_name,
    a.last_name,
    count(fa.film_id) films
FROM
    actor a JOIN
    film_actor fa ON fa.actor_id = a.actor_id
GROUP BY a.actor_id
ORDER BY films DESC
LIMIT 1
""")

,first_name,last_name,films
0,GINA,DEGENERES,42


In [29]:
# Is ‘Academy Dinosaur’ available for rent from Store 1?
q(
"""
SELECT
    f.film_id,
    f.title,
    i.inventory_id,
    s.store_id
FROM
    film f JOIN
    inventory i ON i.film_id = f.film_id
    JOIN
    store s ON s.store_id = i.store_id
WHERE f.title = "ACADEMY DINOSAUR" AND s.store_id = 1;
""")

,film_id,title,inventory_id,store_id
0,1,ACADEMY DINOSAUR,1,1
1,1,ACADEMY DINOSAUR,2,1
2,1,ACADEMY DINOSAUR,3,1
3,1,ACADEMY DINOSAUR,4,1


In [30]:
# Mary Smith's customer id
customer_id = q("""
                SELECT
                    customer_id
                FROM
                    customer
                WHERE first_name || " " || last_name = "MARY SMITH"
                """
               )["customer_id"][0]
# "ACADEMY DINOSAUR"'s inventory id
inventory_id = q("""
                SELECT
                    f.title,
                    i.inventory_id
                FROM
                    film f JOIN
                    inventory i ON i.film_id = f.film_id
                WHERE f.title = "ACADEMY DINOSAUR" and i.store_id=1;
                """
                )["inventory_id"][0]

# Mike Hillyer's staff id
staff_id = q("""
            SELECT
                staff_id,
                first_name,
                last_name
            FROM
                staff
            WHERE first_name || " " || last_name = "Mike Hillyer"
            """
            )["staff_id"][0]

In [32]:
# Insert a record to represent Mary Smith renting ‘Academy Dinosaur’ from Mike Hillyer at Store 1 today .
try:
    q("""
    INSERT INTO rental (
        inventory_id,
        customer_id,
        staff_id
    )
    VALUES (
        {0},
        {1},
        {2}
    )
    """.format(inventory_id, customer_id, staff_id))
except:
    print("data base is read only, therefore error is occuring")

data base is read only, therefore error occured


In [41]:
# What is that average running time of all the films in the sakila DB?
q("""
SELECT
    CAST(AVG(length) AS TEXT) || " minutes" AS average_length
FROM
    film;
""")

,average_length
0,115.272 minutes


In [48]:
# What is the average running time of films by category?
q("""
SELECT
    c.name,
    ROUND(AVG(f.length)) average_length
FROM
    film f JOIN
       film_category fc ON fc.film_id = f.film_id
       JOIN
       category c ON c.category_id = fc.category_id
GROUP BY fc.category_id;
""")

,name,average_length
0,Action,112.0
1,Animation,111.0
2,Children,110.0
3,Classics,112.0
4,Comedy,116.0
5,Documentary,109.0
6,Drama,121.0
7,Family,115.0
8,Foreign,122.0
9,Games,128.0


In [52]:
# Why does this query return the empty set?
print("Both tables have 2 common columns: film_id and last_upadate. There are no matching rows for them")
q("""
select * from film natural join inventory;
""")

Both tables have 2 common columns: film_id and last_upadate. There are no matching rows for them


,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update,inventory_id,store_id


In [ ]:
q("""
SELECT
    c.name,
    AVG(f.length) average_length
FROM
    film f JOIN
       film_category fc ON fc.film_id = f.film_id
       JOIN
       category c ON c.category_id = fc.category_id
GROUP BY fc.category_id;
""")

In [ ]:
# longest per category
q("""
SELECT
    c.name,
    MAX(f.length) average_length
FROM
    film f 
    JOIN
       film_category fc ON fc.film_id = f.film_id
    JOIN
       category c ON c.category_id = fc.category_id
GROUP BY fc.category_id;
""")

In [ ]:
q("""
SELECT
    c.name category,
    f.title,
    f.length
FROM
    film f 
    JOIN
       film_category fc ON fc.film_id = f.film_id
    JOIN
       category c ON c.category_id = fc.category_id
WHERE (fc.category_id, f.length) IN (SELECT
                                        c.category_id,
                                        MAX(f.length) average_length
                                    FROM
                                        film f 
                                        JOIN
                                           film_category fc ON fc.film_id = f.film_id
                                        JOIN
                                           category c ON c.category_id = fc.category_id
                                    GROUP BY fc.category_id);
""")

In [ ]:
q("""
select 
    * 
from 
    film 
        natural join 
    inventory
""")

print("There are no matching rows for common columns: film_id and last_update")

## 2. Excercises for Sakila database from [-Exercises-with-Sakila-DB-](https://github.com/adesai25/MySQL-Exercises-with-Sakila-DB-)

In [53]:
# 1a. Display the first and last names of all actors from the table actor.
q("""
SELECT
    first_name,
    last_name
FROM
    actor;
""")

,first_name,last_name
0,PENELOPE,GUINESS
1,NICK,WAHLBERG
2,ED,CHASE
3,JENNIFER,DAVIS
4,JOHNNY,LOLLOBRIGIDA
...,...,...
195,BELA,WALKEN
196,REESE,WEST
197,MARY,KEITEL
198,JULIA,FAWCETT


In [54]:
# 1b. Display the first and last name of each actor in a single column in upper case letters. Name the column Actor Name.
q("""
SELECT
    UPPER(first_name || " " || last_name) Actor_Name
FROM
    actor;
""")

,Actor_Name
0,PENELOPE GUINESS
1,NICK WAHLBERG
2,ED CHASE
3,JENNIFER DAVIS
4,JOHNNY LOLLOBRIGIDA
...,...
195,BELA WALKEN
196,REESE WEST
197,MARY KEITEL
198,JULIA FAWCETT


In [55]:
# 2a. You need to find the ID number, first name, and last name of an actor,
# of whom you know only the first name, "Joe." What is one query would you use to obtain this information?
q("""
SELECT
    actor_id,
    first_name,
    last_name
FROM
    actor
WHERE first_name = "JOE"
""")

,actor_id,first_name,last_name
0,9,JOE,SWANK


In [56]:
# 2b. Find all actors whose last name contain the letters GEN:
q("""
SELECT
    first_name,
    last_name
FROM
    actor
WHERE last_name LIKE "%GEN%"
""")

,first_name,last_name
0,VIVIEN,BERGEN
1,JODIE,DEGENERES
2,GINA,DEGENERES
3,NICK,DEGENERES


In [57]:
# 2c. Find all actors whose last names contain the letters LI. 
# This time, order the rows by last name and first name, in that order:
q("""
SELECT
    first_name,
    last_name
FROM
    actor
WHERE last_name LIKE "%LI%"
ORDER BY last_name, first_name
""")

,first_name,last_name
0,GREG,CHAPLIN
1,WOODY,JOLIE
2,AUDREY,OLIVIER
3,CUBA,OLIVIER
4,GROUCHO,WILLIAMS
5,MORGAN,WILLIAMS
6,SEAN,WILLIAMS
7,BEN,WILLIS
8,GENE,WILLIS
9,HUMPHREY,WILLIS


In [58]:
#2d. Using IN, display the country_id and country columns of the following countries: 
# Afghanistan, Bangladesh, and China:
q("""
SELECT
    country_id,
    country
FROM
    country
WHERE country IN ("Afghanistan", "Bangladesh", "China")
""")

,country_id,country
0,1,Afghanistan
1,12,Bangladesh
2,23,China


In [ ]:
# inserting deleting cannot be be done on read only database

In [ ]:
q("""
SELECT
    last_name,
    count(last_name)
FROM
    actor
GROUP BY last_name
""")

In [60]:
# 4a. List the last names of actors, as well as how many actors have that last name.
q("""
SELECT
    last_name,
    count(last_name) amount
FROM
    actor
GROUP BY last_name
HAVING amount > 1
ORDER BY count(last_name) DESC
""")

,last_name,amount
0,KILMER,5
1,NOLTE,4
2,TEMPLE,4
3,AKROYD,3
4,ALLEN,3
5,BERRY,3
6,DAVIS,3
7,DEGENERES,3
8,GARLAND,3
9,GUINESS,3


In [ ]:
q("""
SELECT
    first_name,
    last_name
FROM
    actor
WHERE first_name = "GROUCHO" AND last_name = "WILLIAMS"
""")

In [ ]:
q("""
UPDATE actor
SET first_name = "HARPO"
WHERE first_name = "GROUCHO" AND last_name = "WILLIAMS";
""")

In [ ]:
q("""
SELECT
    actor_id,
    first_name,
    last_name
FROM
    actor
WHERE first_name = "GROUCHO" AND last_name = "WILLIAMS"
""")

In [92]:
# 5a. You cannot locate the schema of the address table. Which query would you use to re-create it?
print(q("""
SELECT sql 
FROM sqlite_master 
WHERE name = "address";
""")["sql"].values[0].replace("\r\n", "\n"))

CREATE TABLE address (
  address_id int NOT NULL,
  address VARCHAR(50) NOT NULL,
  address2 VARCHAR(50) DEFAULT NULL,
  district VARCHAR(20) NOT NULL,
  city_id INT  NOT NULL,
  postal_code VARCHAR(10) DEFAULT NULL,
  phone VARCHAR(20) NOT NULL,
  last_update TIMESTAMP NOT NULL,
  PRIMARY KEY  (address_id),
  CONSTRAINT fk_address_city FOREIGN KEY (city_id) REFERENCES city (city_id) ON DELETE NO ACTION ON UPDATE CASCADE
)


In [93]:
# 6a. Use JOIN to display the first and last names, as well as the address, of each staff member.
# Use the tables staff and address:
q("""
SELECT
    s.first_name,
    s.last_name,
    a.address || IFNULL(a.address2, " ") address
FROM
    staff s
        JOIN
    address a ON s.address_id = a.address_id;
""")

,first_name,last_name,address
0,Mike,Hillyer,23 Workhaven Lane
1,Jon,Stephens,1411 Lillydale Drive


In [94]:
# 6b. Use JOIN to display the total amount rung up by each staff member in August of 2005. 
# Use tables staff and payment.
q("""
SELECT
    (s.first_name || " " || s.last_name) name,
    SUM(p.amount) amount,
    STRFTIME("%Y", p.payment_date) year,
    STRFTIME("%m", p.payment_date) month
FROM
    staff s
        JOIN
    payment p ON p.staff_id = s.staff_id
WHERE year = "2005" AND month = "08"
GROUP BY name;
""")

,name,amount,year,month
0,Jon Stephens,12218.48,2005,08
1,Mike Hillyer,11853.65,2005,08


In [95]:
# 6c. List each film and the number of actors who are listed for that film. 
# Use tables film_actor and film. Use inner join.
q("""
SELECT
    f.title,
    COUNT(fa.actor_id) actors_num
FROM
    film f
        JOIN
    film_actor fa ON fa.film_id = f.film_id
GROUP BY f.title
ORDER BY actors_num DESC
""")

,title,actors_num
0,LAMBS CINCINATTI,15
1,BOONDOCK BALLROOM,13
2,CHITTY LOCK,13
3,CRAZY HOME,13
4,DRACULA CRYSTAL,13
...,...,...
992,PIRATES ROXANNE,1
993,PSYCHO SHRUNK,1
994,SHOCK CABIN,1
995,STONE FIRE,1


In [102]:
# 6d. How many copies of the film Hunchback Impossible exist in the inventory system?
q("""
SELECT
    f.title,
    COUNT(f.title) copies
FROM
    film f 
        JOIN
    inventory i ON i.film_id=f.film_id
WHERE f.title = "HUNCHBACK IMPOSSIBLE"
""")

,title,copies
0,HUNCHBACK IMPOSSIBLE,6


In [96]:
# 6e. Using the tables payment and customer and the JOIN command, list the total paid by each customer.
# List the customers alphabetically by last name:
q("""
SELECT
    (c.first_name || " "|| c.last_name) full_name,
    SUM(p.amount)
FROM
    customer c
        JOIN
    payment p ON p.customer_id = c.customer_id
GROUP BY full_name
ORDER BY c.last_name
""")

,full_name,SUM(p.amount)
0,RAFAEL ABNEY,97.79
1,NATHANIEL ADAM,133.72
2,KATHLEEN ADAMS,92.73
3,DIANA ALEXANDER,105.73
4,GORDON ALLARD,160.68
...,...,...
594,BRENDA WRIGHT,104.74
595,BRIAN WYMAN,52.88
596,LUIS YANEZ,79.80
597,MARVIN YEE,75.79


In [103]:
# 7a. The music of Queen and Kris Kristofferson have seen an unlikely resurgence. 
# As an unintended consequence, films starting with the letters K and Q have also soared in popularity. 
# Use subqueries to display the titles of movies starting with the letters K and Q whose language is English.
q("""
SELECT
    title
FROM
    (SELECT
        f.title,
        l.name language
    FROM
        film f
            JOIN
        language l ON l.language_id = f.language_id
    WHERE l.name = "English")
WHERE title LIKE "Q%" OR title LIKE "K%"
""")

,title
0,KANE EXORCIST
1,KARATE MOON
2,KENTUCKIAN GIANT
3,KICK SAVANNAH
4,KILL BROTHERHOOD
5,KILLER INNOCENT
6,KING EVOLUTION
7,KISS GLORY
8,KISSING DOLLS
9,KNOCK WARLOCK


In [104]:
# 7b. Use subqueries to display all actors who appear in the film Alone Trip.
q("""
SELECT
    actor
FROM
    (SELECT 
        a.first_name || " " || a.last_name actor,
        f.title
    FROM
        actor a
            JOIN
        film_actor fa ON fa.actor_id = a.actor_id
            JOIN
        film f ON f.film_id = fa.film_id)
WHERE title = "ALONE TRIP"
""")

,actor
0,ED CHASE
1,KARL BERRY
2,UMA WOOD
3,WOODY JOLIE
4,SPENCER DEPP
5,CHRIS DEPP
6,LAURENCE BULLOCK
7,RENEE BALL


In [105]:
# 7c. You want to run an email marketing campaign in Canada,
# for which you will need the names and email addresses of all Canadian customers. 
# Use joins to retrieve this information.
q("""
SELECT
    cu.first_name || " " || cu.last_name name,
    cu.email,
    co.country
FROM
    customer cu
        JOIN
    address a ON a.address_id = cu.address_id
        JOIN
    city ci ON ci.city_id = a.city_id
        JOIN
    country co ON co.country_id = ci.country_id
WHERE country = "Canada"
""")

,name,email,country
0,LORETTA CARPENTER,LORETTA.CARPENTER@sakilacustomer.org,Canada
1,CURTIS IRBY,CURTIS.IRBY@sakilacustomer.org,Canada
2,TROY QUIGLEY,TROY.QUIGLEY@sakilacustomer.org,Canada
3,DARRELL POWER,DARRELL.POWER@sakilacustomer.org,Canada
4,DERRICK BOURQUE,DERRICK.BOURQUE@sakilacustomer.org,Canada


In [106]:
# 7d. Sales have been lagging among young families, and you wish to target all family movies for a promotion. 
# Identify all movies categorized as famiy films.
q("""
SELECT
    f.title,
    c.name
FROM
    film f
        JOIN
    film_category fc ON fc.film_id = f.film_id
        JOIN
    category c ON c.category_id = fc.category_id
WHERE name = "Family"
""")

,title,name
0,AFRICAN EGG,Family
1,APACHE DIVINE,Family
2,ATLANTIS CAUSE,Family
3,BAKED CLEOPATRA,Family
4,BANG KWAI,Family
...,...,...
64,SPLASH GUMP,Family
65,SUNSET RACER,Family
66,SUPER WYOMING,Family
67,VIRTUAL SPOILERS,Family


In [107]:
# 7e. Display the most frequently rented movies in descending order.
q("""
SELECT
    f.title,
    COUNT(r.rental_id) rent_num
FROM
    film f
        JOIN
    inventory i ON i.film_id = f.film_id
        JOIN
    rental r ON r.inventory_id = i.inventory_id
GROUP BY f.title
ORDER BY rent_num DESC
""")

,title,rent_num
0,BUCKET BROTHERHOOD,34
1,ROCKETEER MOTHER,33
2,FORWARD TEMPLE,32
3,GRIT CLOCKWORK,32
4,JUGGLER HARDLY,32
...,...,...
953,SEVEN SWARM,5
954,TRAFFIC HOBBIT,5
955,HARDLY ROBBERS,4
956,MIXED DOORS,4


In [108]:
# 7f. Write a query to display how much business, in dollars, each store brought in.
q("""
SELECT
    s.store_id,
    SUM(p.amount)
FROM
    payment p
        JOIN
    staff s ON s.staff_id = p.staff_id
        JOIN
    store st ON st.store_id = s.store_id
GROUP BY s.store_id
""")

,store_id,SUM(p.amount)
0,1,33489.47
1,2,33927.04


In [109]:
# 7g. Write a query to display for each store its store ID, city, and country.
q("""
SELECT
    s.store_id,
    ci.city,
    co.country
FROM
    store s
        JOIN
    address a ON a.address_id = a.address_id
        JOIN
    city ci ON ci.city_id = a.city_id
        JOIN
    country co ON co.country_id = ci.country_id
WHERE s.address_id = a.address_id;
""")

,store_id,city,country
0,1,Lethbridge,Canada
1,2,Woodridge,Australia


In [110]:
# 7h. List the top five genres in gross revenue in descending order. 
# (Hint: you may need to use the following tables: category, film_category, inventory, payment, and rental.)
q("""
SELECT
    c.name,
    SUM(p.amount) total_revenue
FROM
    rental r
        JOIN
    inventory i ON i.inventory_id = r.inventory_id
        JOIN
    film_category fc ON fc.film_id = i.film_id
        JOIN
    category c ON c.category_id = fc.category_id
        JOIN
    payment p ON p.rental_id = r.rental_id
GROUP BY name
ORDER BY total_revenue DESC
LIMIT 5;
""")

,name,total_revenue
0,Sports,5314.21
1,Sci-Fi,4756.98
2,Animation,4656.30
3,Drama,4587.39
4,Comedy,4383.58


In [112]:
# 8a. In your new role as an executive, 
# you would like to have an easy way of viewing the Top five genres by gross revenue. 
# Use the solution from the problem above to create a view. 
# f you haven't solved 7h, you can substitute another query to create a view.
try:
    q("""
    CREATE VIEW v_top_five_genres
    AS
    SELECT
        c.name,
        SUM(p.amount) total_revenue
    FROM
        rental r
            JOIN
        inventory i ON i.inventory_id = r.inventory_id
            JOIN
        film_category fc ON fc.film_id = i.film_id
            JOIN
        category c ON c.category_id = fc.category_id
            JOIN
        payment p ON p.rental_id = r.rental_id
    GROUP BY name
    ORDER BY total_revenue DESC
    LIMIT 5;
    """)
except:
    print("read only database")

read only database


In [114]:
# 8b. How would you display the view that you created in 8a?
q("""
SELECT
    *
FROM
    sales_by_film_category
ORDER BY total_sales DESC
LIMIT 5;
""")

,category,total_sales
0,Sports,5314.21
1,Sci-Fi,4756.98
2,Animation,4656.30
3,Drama,4587.39
4,Comedy,4383.58


In [ ]:
# 8c. You find that you no longer need the view top_five_genres. 
# Write a query to delete it.
try:
    q("""
    DROP VIEW v_top_five_genres;
    """)
except:
    print("read only database")